In [1]:
import numpy as np

In [2]:
def parse_sample(sample_name: str) -> str:
    sample = open(f"samples/{sample_name}.txt")
    parsed_sample = sample.read().replace("\n", "")
    return parsed_sample

In [163]:
def needleman_wunsh(sample1, sample2, match=2, mismatch=-2, gap=-3) -> tuple[int, np.ndarray]:
    
    matrix = np.zeros([len(sample2)+1, len(sample1)+1], int)
    matrix[0, 1:] = [-3 * (idx+1) for idx, _ in enumerate(sample1)]
    matrix[1:, 0] = [-3 * (idx+1) for idx, _ in enumerate(sample2)]
    for i in range(1, matrix.shape[0]):
        for j in range(1, matrix.shape[1]):
            matrix[i, j] = max(
                                matrix[i-1, j-1] + 
                                    (match if sample2[i-1] == sample1[j-1] 
                                     else mismatch),
                                matrix[i-1, j] + gap,
                                matrix[i, j-1] + gap
                              )
    
    return matrix[-1, -1], matrix
    

In [164]:
def get_backtracking(sample1, sample, matrix: np.ndarray) -> list[list[str]]:
    sample1_aligned = []
    sample2_aligned = []
    match_mismatch = []
    i, j = matrix.shape
    i -= 1
    j -= 1
    direcao = []
    while True:
        #print(f"amostra1: {sample1[j-1]} amostra2: {sample2[i-1]}", end="    ")
        if i > 0 and j > 0:
            upper = matrix[i-1, j]
            diagonal = matrix[i-1, j-1]
            left = matrix[i, j-1]

            if sample1[j-1] == sample2[i-1]:
                sample1_aligned.insert(0, sample1[j-1])
                sample2_aligned.insert(0, sample2[i-1])
                match_mismatch.insert(0, "*")

                i -= 1
                j -= 1

                #print(f"caso 0 esquerda:{left} cima:{upper} diagonal: {diagonal}   direcao: diagonal")

            elif  (diagonal >= left and diagonal >= upper):
                sample1_aligned.insert(0, sample1[j-1])
                sample2_aligned.insert(0, sample2[i-1])
                match_mismatch.insert(0, "|")

                i -= 1
                j -= 1

                #print(f"caso 1 esquerda:{left} cima:{upper} diagonal: {diagonal}   direcao: diagonal")


            elif (left > diagonal and left > upper):
                sample1_aligned.insert(0, sample1[j-1])
                sample2_aligned.insert(0, "-")
                match_mismatch.insert(0, " ")

                j -= 1
                #print(f"caso 2 esquerda:{left} cima:{upper} diagonal: {diagonal}   direcao: esquerda")

            else:
                sample1_aligned.insert(0, "-")
                sample2_aligned.insert(0, sample2[i-1])
                match_mismatch.insert(0, " ")

                i -= 1
                direcao.append("U")

                #print(f"caso 3 esquerda:{left} cima:{upper} diagonal: {diagonal}   direcao: cima")

        elif i > 0:
                left = matrix[i, j-1]
                sample1_aligned.insert(0, sample1[j-1])
                sample2_aligned.insert(0, "-")
                match_mismatch.insert(0, " ")

                j -= 1
                direcao.append("L")

                #print(f"caso 4 esquerda:{left}  direcao: esquerda")

        elif i > 0:

            upper = matrix[i-1, j]
            sample1_aligned.insert(0, "-")
            sample2_aligned.insert(0, sample2[i-1])
            match_mismatch.insert(0, " ")

            i -= 1
            direcao.append("U")

            #print(f"caso 5 cima:{upper} direcao: cima")

        else:
            break
            
            
    return sample1_aligned, match_mismatch, sample2_aligned

In [169]:
seqs = [("korea", "porto_rico"), 
        ("korea", "guangdong"),
        ("porto_rico", "guangdong")]
for sample1_name, sample2_name in seqs:
    sample1 = parse_sample(sample1_name)
    sample2 = parse_sample(sample2_name)
    score, matrix = needleman_wunsh(sample1, sample2)
    
    sample1_aligned, match_mismatch, sample2_aligned = get_backtracking(sample1, sample2, matrix)
    
    identity = int(match_mismatch.count("*") / len(match_mismatch) * 100)
    print(f"Amostra 1: {sample1_name} | Amostra 2: {sample2_name} | Score: {score} | Identity: {identity}%")
    
    
    with open(f"outputs/{sample1_name}_{sample2_name}.txt", "w") as file:
        file.write(''.join(sample1_aligned))
        file.write('\n')
        file.write(''.join(match_mismatch))
        file.write('\n')
        file.write(''.join(sample2_aligned))
        

Amostra 1: korea | Amostra 2: porto_rico | Score: 1198 | Identity: 69%
Amostra 1: korea | Amostra 2: guangdong | Score: 1380 | Identity: 71%
Amostra 1: porto_rico | Amostra 2: guangdong | Score: 1048 | Identity: 67%
